In [1]:
import os

from langchain.document_loaders import PyPDFLoader, PDFMinerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from datasets import load_dataset_builder, load_dataset



In [11]:
# Get the list of all files and directories
ct = 0
resume_pdfs_path = "data/data/"
file_paths_ls = []
for (root, dirs, file) in os.walk(resume_pdfs_path):
    for f in file:
        ct += 1

print(ct)

2484


#### Extracting key documnets from pdfs

In [2]:
# Get the list of all files and directories
def get_files():
    resume_pdfs_path = "test_resumes/"
    file_paths_ls = []
    for (root, dirs, file) in os.walk(resume_pdfs_path):
        # if root == resume_pdfs_path + "SALES":
        for pdf in file:
            f = pdf
            file_path = root + f
            file_paths_ls.append(file_path)
                # print(file_path)

    return file_paths_ls
    # print(file_paths_ls)

In [3]:
files_path = get_files()
files_path

['test_resumes/acc.pdf',
 'test_resumes/advo.pdf',
 'test_resumes/agri.pdf',
 'test_resumes/apparel.pdf',
 'test_resumes/arts.pdf',
 'test_resumes/automobile.pdf',
 'test_resumes/aviation.pdf',
 'test_resumes/bank.pdf',
 'test_resumes/bpo.pdf',
 'test_resumes/bus_dev.pdf',
 'test_resumes/chef.pdf',
 'test_resumes/construction.pdf',
 'test_resumes/consultant.pdf',
 'test_resumes/designer.pdf',
 'test_resumes/dig_media.pdf',
 'test_resumes/engg.pdf',
 'test_resumes/finance.pdf',
 'test_resumes/fitness.pdf',
 'test_resumes/healthcare.pdf',
 'test_resumes/hr.pdf',
 'test_resumes/it.pdf',
 'test_resumes/pr.pdf',
 'test_resumes/sales.pdf',
 'test_resumes/teacher.pdf']

In [4]:

def page_extractor(files_path):
    # dict of lists
    pdfs_dict = {}
    for file in files_path:
        file_path_key = file[10:]
        pdfs_dict[file_path_key] = []
        loader = PDFMinerLoader(file)

        # Load Documents and split into chunks. Chunks are returned as Documents.
        pages = loader.load_and_split(text_splitter=RecursiveCharacterTextSplitter(chunk_size = 4000, chunk_overlap=0))
        # print(pages[0].page_content)
        for page in pages:
            data = page.page_content
            pdfs_dict[file_path_key].append(data)
            # print(data)
            # print('**************************************')

        # print('------------------------------')
    return pdfs_dict

In [5]:
pdfs_dict = page_extractor(files_path)
print(pdfs_dict)

{'es/acc.pdf': ['ACCOUNTANT\nSummary\n\nSelf-motivated accountant offering a strong work ethic and determination to complete tasks in a timely manner. Accurate and detail-oriented with\nextensive auditing and finance knowledge.\n\nHighlights\n\nComplex problem solving\nStrong communication skills\nExpert in customer relations\nPortfolio management\n\nA\n\nAccomplishments\n\nProficient in Microsoft Office\nMicrosoft Excel expert\nRisk management expertise\n\nFinancial statement analysis\n\nGeneral ledger accounting\n\nAchieved an internal audit score of 95%.\n\nPromoted to Auditor after just 3 months of employment.\n\nBoosted quality ratings by 85% by developing new processes and improving work flow.\n\nExperience\nAccountant \nCurrent to September 2014 Company Name ï¼\u200b City , State\n\nPrepare unpaid reports on actual expenses for marketing line of business.\nCreate and maintain pending and process able database.\nPrepare and setup vendor purchase orders contracts as well as CRX te

In [6]:
def get_skills_and_edu(pdfs_dict: dict) -> dict:
    
    """ Extracts skills and education corresponding to each pdf """

    pdfs_relevant = {}
    for key, value in pdfs_dict.items(): # value -> list of page contents
        pdfs_relevant[key] = "" # for storing skills and education
        skill_flag = edu_flag = False
        for page in value:
            for each_line in page.split('\n'):
                # print(each_line)
                line_ls = each_line.lower().split(' ')
                
                if skill_flag and len(line_ls) <= 3 and ("experience" in line_ls or ("work" in line_ls and "history" in line_ls) or "education" in line_ls):
                    skill_flag = False
                if edu_flag and len(line_ls) <= 3 and ("experience" in line_ls or ("work" in line_ls and "history" in line_ls) or "skills" in line_ls):
                    edu_flag = False

                if skill_flag and each_line != '\n':
                    pdfs_relevant[key] += each_line + "\n"
                if edu_flag and each_line != '\n':
                    pdfs_relevant[key] += each_line + "\n"

                if len(line_ls) <= 3 and "skills" in line_ls: # Accounting for variations in heading 
                    skill_flag = True
                if len(line_ls) <= 3 and "education" in line_ls:
                    edu_flag = True
                
                
    return pdfs_relevant

In [7]:
pdfs_relevant = get_skills_and_edu(pdfs_dict)
for k, v in pdfs_relevant.items():
    print(v)
    print('---------------------------------')

Expert in customer relations
Portfolio management

A

Accomplishments

Proficient in Microsoft Office
Microsoft Excel expert
Risk management expertise

Financial statement analysis

General ledger accounting

Achieved an internal audit score of 95%.

Promoted to Auditor after just 3 months of employment.

Boosted quality ratings by 85% by developing new processes and improving work flow.

Accounting Certificate : Accounting , 2012 Cecil County Community College ï¼​ City , State
Performing Payroll in QuickBooks 2009 Certificate : Payroll , 2012 Cecil County Community College ï¼​ City , State
Bachelor of Science : General Studies , 2011 Wilmington University ï¼​ City , State , New Castle County

General Studies


Accounting, balance, budget, business analyst, Call Center, Cash Management, closing, contracts, Critical Thinking, client, clients, databases,
database, documentation, Financial, financial statements, leadership, Team Lead, marketing, excel, mail, MS Office Suites, Payroll, pol

#### Extracting details from job descriptions

In [8]:
from datasets import load_dataset

dataset = load_dataset("jacob-hugging-face/job-descriptions", split="train")

In [9]:
# Dataset to dataframes

import pandas as pd

dataset.set_format(type='pandas')
job_ds_features = dataset[:]

In [10]:
job_ds_features.head()

,company_name,job_description,position_title,description_length,model_response
0,Google,minimum qualifications\nbachelors degree or eq...,Sales Specialist,2727,"{\n ""Core Responsibilities"": ""Responsible fo..."
1,Apple,description\nas an asc you will be highly infl...,Apple Solutions Consultant,828,"{\n ""Core Responsibilities"": ""as an asc you ..."
2,Netflix,its an amazing time to be joining netflix as w...,Licensing Coordinator - Consumer Products,3205,"{\n ""Core Responsibilities"": ""Help drive bus..."
3,Robert Half,description\n\nweb designers looking to expand...,Web Designer,2489,"{\n ""Core Responsibilities"": ""Designing webs..."
4,TrackFive,at trackfive weve got big goals were on a miss...,Web Developer,3167,"{\n ""Core Responsibilities"": ""Build and layo..."


In [11]:
# get Job descriptions

import random

def get_job_profiles(num_of_jobs=5, where='top'):

    """ returns num_of_jobs job profiles from where location in dataset"""
    if where == "top":
        return job_ds_features.iloc[:num_of_jobs]
    elif where == "bottom":
        return job_ds_features.iloc[-num_of_jobs:]
    elif where == "shuffle": # random
        idx = random.choices(range(0, len(job_ds_features)), k=num_of_jobs)
        return job_ds_features.iloc[idx]

    

In [53]:
n_jds = get_job_profiles(5, "shuffle")
n_jds

,company_name,job_description,position_title,description_length,model_response
1549,Public Consulting Group United States Remote,senior project manager education\njob locatio...,Senior Project Manager - Education,3387,"{\n ""Core Responsibilities"": ""Serve as a pro..."
860,Zander Insurance Agency,job description\n\nzander insurance group is o...,Web Designer,2896,"{\n ""Core Responsibilities"": ""Design compell..."
480,"SBM Management Services, LP",description\n\nposition at sbm management\n\ns...,Site Administrative Assistant,1842,"{\n ""Core Responsibilities"": ""Support site m..."
1289,Vera Institute of Justice,who we are\n\nthe vera institute of justice fo...,"Systems Administrator, IT (Brooklyn, NY)",6878,"{\n ""Core Responsibilities"": ""Responsible fo..."
1054,"Magic Transport, Inc",company profile freight forwarder consolidato...,General Office Clerk / Data Entry,478,"{\n ""Core Responsibilities"": ""General office..."


In [54]:
len(n_jds['job_description'].iloc[0])

3387

#### Chunking 

In [14]:
# splitting into small texts can also improve vector store search results, as smaller segments might be more likely to match query
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
)


In [55]:
resumes_data = list(pdfs_relevant.values())
resumes_metadata = [{'doc_src': src} for src in pdfs_relevant.keys()]

resume_documents = text_splitter.create_documents(resumes_data, metadatas=resumes_metadata)

In [56]:
resume_documents

[Document(page_content='Expert in customer relations\nPortfolio management\n\nA\n\nAccomplishments\n\nProficient in Microsoft Office\nMicrosoft Excel expert\nRisk management expertise\n\nFinancial statement analysis\n\nGeneral ledger accounting\n\nAchieved an internal audit score of 95%.\n\nPromoted to Auditor after just 3 months of employment.\n\nBoosted quality ratings by 85% by developing new processes and improving work flow.\n\nAccounting Certificate : Accounting , 2012 Cecil County Community College ï¼\u200b City , State\nPerforming Payroll in QuickBooks 2009 Certificate : Payroll , 2012 Cecil County Community College ï¼\u200b City , State\nBachelor of Science : General Studies , 2011 Wilmington University ï¼\u200b City , State , New Castle County\n\nGeneral Studies', metadata={'doc_src': 'es/acc.pdf'}),
 Document(page_content='Accounting, balance, budget, business analyst, Call Center, Cash Management, closing, contracts, Critical Thinking, client, clients, databases,\ndatabase,

In [134]:
## Chunking job_descriptions
job_des = list(n_jds['job_description'])
model_response = list(n_jds['model_response'])

jobs_data = []
for i in range(len(job_des)):
    needed_skills = model_response[i].split('\n')
    needed_skills = "\n".join([ns.strip(",") for ns in needed_skills][1:4])
    complete_job_des = needed_skills + "\n" + job_des[i] 
    
    jobs_data.append(complete_job_des)

jobs_metadata = [{'company': comp, 'position': pos} for comp, pos in tuple(n_jds.apply(lambda row: (row['company_name'], row['position_title']), axis=1))]

job_documents = text_splitter.create_documents(jobs_data, metadatas=jobs_metadata)

In [133]:
# for doc in resume_documents:
#     print(doc)
#     print('------------------------')

print('*****************************')
for doc in jobs_data:
    print(doc)
    print('------------------------')


*****************************
  "Core Responsibilities": "Serve as a project manager across multiple special education professional development and consulting projects with local and state clients. Develops mechanisms for monitoring project progress and problem solving with content leads and client representatives. Oversees and coordinates multiple projects across the team to ensure each client's deliverables are met in a timely manner. Reviews status of projects and budgets, manages schedules, and prepares status reports. Investigates and analyzes clients processes, recommends solutions to meet the client's needs. Manages team client training schedule to optimize capacity and timelines, and collaborates with team to plan and execute new projects. Assigns tasks to project team members in collaboration with supervisor, monitors progress and completion, and reports any delays in delivery. Oversees operational aspects of ongoing projects including monitoring the team's business developmen

In [135]:
print(len(job_documents))

29


#### Embedding jobs and resumes

##### Sentence Transformers

In [136]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

# Sentence Transformer Embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# Equivalent to SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [137]:
# Generate embeddings for the texts
job_embedding = embeddings.embed_documents([jobs_data[1]])

# Print the embeddings
# print(f"Text: {jobs_data[0]}")

In [138]:
import numpy as np

np.shape(job_embedding)
# job_embedding

(1, 384)

In [139]:
# Embeddings all resumes:
query_res = [relv_data.page_content for relv_data in resume_documents]
query_meta = [meta.metadata['doc_src'] for meta in resume_documents]


In [140]:
# Generate embeddings for the texts
cv_embedding = embeddings.embed_documents(query_res)

# Print the embeddings
# print(f"Text: {query_res}")

In [141]:
import numpy as np

np.shape(cv_embedding)
# job_embedding

(43, 384)

##### BERT embeddings

In [144]:
import torch
from transformers import BertTokenizer, BertModel

model_id = "bert-base-uncased"

model = BertModel.from_pretrained(model_id)
tokenizer = BertTokenizer.from_pretrained(model_id)


In [145]:
# Encoding resumes and job with bert
# initialize dictionary to store tokenized sentences
resume_tokens = {'input_ids': [], 'attention_mask': []}


for resume in query_res:
    # encode each sentence and append to dictionary
    new_tokens = tokenizer.encode_plus(resume, max_length=128,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')
    resume_tokens['input_ids'].append(new_tokens['input_ids'][0])
    resume_tokens['attention_mask'].append(new_tokens['attention_mask'][0])



# reformat list of tensors into single tensor
resume_tokens['input_ids'] = torch.stack(resume_tokens['input_ids'])
resume_tokens['attention_mask'] = torch.stack(resume_tokens['attention_mask'])

resume_outputs = model(**resume_tokens)

print(resume_outputs.keys())


odict_keys(['last_hidden_state', 'pooler_output'])


In [146]:

job_tokens = tokenizer.encode_plus(jobs_data[1], max_length=128,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')

# print(job_tokens)
job_outputs = model(**job_tokens)
print(job_outputs.keys())


odict_keys(['last_hidden_state', 'pooler_output'])


In [147]:
resume_embeddings = resume_outputs.last_hidden_state
job_embeddings = job_outputs.last_hidden_state
# resume_embeddings

In [148]:
print(resume_embeddings.shape)
print(job_embeddings.shape)

torch.Size([43, 128, 768])
torch.Size([1, 128, 768])


In [150]:
# Mean pooling to get single representation for a sentence

# Resizing attention mask
resume_attention_mask = resume_tokens['attention_mask']
resume_mask = resume_attention_mask.unsqueeze(-1).expand(resume_embeddings.size()).float()

resume_masked_embeddings = resume_embeddings * resume_mask
resume_masked_embeddings.shape

torch.Size([43, 128, 768])

In [151]:
job_attention_mask = job_tokens['attention_mask']
job_mask = job_attention_mask.unsqueeze(-1).expand(job_embeddings.size()).float()

job_masked_embeddings = job_embeddings * job_mask
job_masked_embeddings.shape

torch.Size([1, 128, 768])

In [152]:
summed_r = torch.sum(resume_masked_embeddings, 1)
summed_mask_r = torch.clamp(resume_mask.sum(1), min=1e-9)

resume_mean_pooled_embeddings = summed_r / summed_mask_r
print(resume_mean_pooled_embeddings.shape)

torch.Size([43, 768])


In [153]:
summed_j = torch.sum(job_masked_embeddings, 1)
summed_mask_j = torch.clamp(job_mask.sum(1), min=1e-9)

job_mean_pooled_embeddings = summed_j / summed_mask_j
print(job_mean_pooled_embeddings.shape)

torch.Size([1, 768])


##### InstructEmbeddings

In [156]:
from langchain.embeddings import HuggingFaceInstructEmbeddings


model_name = "hkunlp/instructor-large"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

embeddings = HuggingFaceInstructEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

load INSTRUCTOR_Transformer
max_seq_length  512


In [157]:
# Generate embeddings for the texts
job_embedding = embeddings.embed_documents([jobs_data[1]])

# Print the embeddings
# print(f"Text: {jobs_data[0]}")

In [158]:
# Embeddings all resumes:
query_res = [relv_data.page_content for relv_data in resume_documents]
query_meta = [meta.metadata['doc_src'] for meta in resume_documents]

In [159]:
# Generate embeddings for the texts
cv_embedding = embeddings.embed_documents(query_res)

# Print the embeddings
# print(f"Text: {query_res}")

In [160]:
import numpy as np

print(np.shape(job_embedding))
print(np.shape(cv_embedding))
# job_embedding

(1, 768)
(43, 768)


#### Similarity matching

In [163]:
# Match each CV against n resume
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Calculate similarity scores
similarity_scores = cosine_similarity(job_embedding, cv_embedding)[0]
print(similarity_scores)



[0.83209389 0.84008905 0.80128978 0.76346352 0.79149815 0.783565
 0.83546989 0.7976677  0.78086069 0.82798482 0.80825212 0.75976762
 0.84277799 0.80369488 0.80954649 0.73459199 0.74898539 0.738497
 0.81571472 0.81142616 0.76925026 0.7942634  0.82952849 0.83810556
 0.82297856 0.7892056  0.84154625 0.78086937 0.80213342 0.79911278
 0.83988256 0.84354714 0.84254423 0.78250539 0.85651443 0.83198935
 0.81796618 0.80931928 0.78571915 0.82017046 0.81099634 0.81432609
 0.81180701]


In [ ]:
# Match each CV against n resume
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


job_mean_pooled_embeddings = job_mean_pooled_embeddings.detach().numpy()
resume_mean_pooled_embeddings = resume_mean_pooled_embeddings.detach().numpy()

# Calculate similarity scores
similarity_scores = cosine_similarity(job_mean_pooled_embeddings, resume_mean_pooled_embeddings)[0]
print(similarity_scores)



In [164]:
# Find the most similar document
most_similar_index = np.argmax(similarity_scores)
most_similar_resume = query_meta[most_similar_index]

print("Most matching resume for the role of {0} at {1}: ".format(jobs_metadata[1]["position"], jobs_metadata[1]["company"]))
print(most_similar_resume)

Most matching resume for the role of Web Designer at Zander Insurance Agency: 
es/hr.pdf
